# Project: Movie Recommender System

In [70]:
import os
os.getcwd()



'c:\\Users\\valar\\Documents\\Flatiron\\Assignments\\Phase 4\\Project\\Phase-4-Project---Movie-Recommendation-System'

In [71]:
os.chdir(r'c:\Users\valar\Documents\Flatiron\Assignments\Phase 4\Project\Phase-4-Project---Movie-Recommendation-System')

In [72]:
%pwd


'c:\\Users\\valar\\Documents\\Flatiron\\Assignments\\Phase 4\\Project\\Phase-4-Project---Movie-Recommendation-System'

# EDA

In [73]:
# load the dataset
import pandas as pd
import numpy as np

df = pd.read_csv('./data/ratings.csv')
df.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [75]:
# dropping the timestamp column as we don't need it
df = df.drop(['timestamp'],axis=1)

In [76]:
# checking that there are no null values in our dataset
df.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [77]:
mov_df = pd.read_csv('./data/movies.csv')
mov_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [78]:
tags_df = pd.read_csv('./data/tags.csv')
tags_df.shape

(3683, 4)

In [79]:
links_df = pd.read_csv('./data/links.csv')
links_df.shape

(9742, 3)

In [80]:
combined_df = pd.merge(df,mov_df,on= 'movieId')
combined_df.shape

(100836, 5)

In [81]:
combined_df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [82]:
# changing out df into the surprise 
# importing surprise
from surprise import Reader,Dataset
reader = Reader()
df_surprise = Dataset.load_from_df(df,reader)

### Doing neighbour-based similarity models
Looking at how many user's vs items we have. 

In [83]:
dataset = df_surprise.build_full_trainset()
print('Number of users',dataset.n_users,'\n')
print('Number of items',dataset.n_items)

Number of users 610 

Number of items 9724


In [84]:
# importing necessary libraries
from surprise.model_selection import cross_validate,GridSearchCV
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNBaseline,KNNWithMeans,KNNBasic


# Evaluating Model with SVD

In [85]:
# 
# param_grid = {'n_factors': [20,50,80],
#               'reg_all':[0.02,0.05],
#               'n_epochs':[15,20]}
# # Instantiate GridsearchCv 
# grid_svd = GridSearchCV(SVD,param_grid=param_grid,measures=['rmse'])

# # fitting to surprise dataset
# grid_svd.fit(df_surprise)


In [86]:
# print('Best Parameters:', grid_svd.best_params)
# print('Best Score', grid_svd.best_score)

Comparing our SVD model with KNNBasic,KNNBaseline,and KNNwithMeans

In [87]:

# cross-validating with KNNBasic
Basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic = cross_validate(Basic,df_surprise,n_jobs=-1,measures=['rmse'])

In [88]:
Basic_trmse = np.mean(cv_knn_basic['test_rmse'])
print('Average test RMSE is:', Basic_trmse)

Average test RMSE is: 0.9740505452871195


KNNBaseline

In [89]:
Baseline = KNNBaseline(sim_options={'name':"pearson",'user_based': True})
cv_knn_baseline = cross_validate(Baseline,df_surprise,n_jobs=-1,measures = ['rmse'])

Baseline_trmse = np.mean(cv_knn_baseline['test_rmse'])
print('Average test RMSE is:',Baseline_trmse )

Average test RMSE is: 0.8762852901373155


# KNNwithMeans

In [90]:
KwMeans = KNNWithMeans(sim_options={'name':"pearson",'user_based': True})
cv_knn_wMeans = cross_validate(KwMeans,df_surprise,n_jobs=-1,measures = ['rmse'])

kwMeans_trmse = np.mean(cv_knn_wMeans['test_rmse'])
print('Average test RMSE is:', kwMeans_trmse)

Average test RMSE is: 0.8979099531232743


In [91]:
Summary = [{'Model': 'SVD', 'Best RMSE': grid_svd.best_score},
           {'Model': 'KNNBasic', 'Best RMSE': Basic_trmse},
           {'Model': 'KNNBaseline', 'Best RMSE': Baseline_trmse},
           {'Model': 'KNNWithMeans', 'Best RMSE': kwMeans_trmse},]

df_summary = pd.DataFrame(Summary)
df_summary

,Model,Best RMSE
0,SVD,{'rmse': 0.8690494238648709}
1,KNNBasic,0.974051
2,KNNBaseline,0.876285
3,KNNWithMeans,0.89791


# Making Movie Recommendations with SVD
SVD had the lowest rmse score at 0.86. We are going to fit and make predictions on this model.

In [92]:
# Instantiat an SVD model
# from GridSearch, Best Params: {'n_factors': 50, 'reg_all': 0.05, 'n_epochs': 20}}

svd = SVD(n_factors=50,reg_all=0.05,n_epochs=20)
svd.fit(dataset)

In [93]:
# Example prediction
svd.predict(1,3)

Prediction(uid=1, iid=3, r_ui=None, est=3.9438901824142456, details={'was_impossible': False})

In [94]:
type(df_surprise)

surprise.dataset.DatasetAutoFolds

The estimated rating for user id 1, movieid 3 is 3.990 which is very close to its true rating = 4.0

In [95]:
# Created a helper function to get a rating list specific to one user
# def get_ratings_list(user_id):
#     if user_id not in df['userId'].values:
#         raise ValueError(f"userId not found in dataset!")
#     r_list = list(df.loc[df['userId'] == user_id, ['movieId','rating']].itertuples(index=False,name=None))
#     return r_list

In [96]:
# get_ratings_list()

In [110]:
# this function takes in a user_id, df_ratings(surprise dataset), and a number indicating how many 
# movie recommendations you want.
def movie_recs(user_id,df=df,num = 5):
    rating_list = []
    if user_id not in df['userId'].values:
         raise ValueError(f"userId not found in dataset!")
    for m_id in df['movieId'].unique():
        rating_list.append((m_id,svd.predict(user_id,m_id)[3]))
    
    ranked_list = sorted(rating_list,key=lambda x:x[1],reverse=True)[:num]
    # from sorted ranked list get movie_title using movie_id
    
    for i,val in enumerate(ranked_list):
        title = mov_df.loc[mov_df['movieId'] == val[0],'title'].iloc[0]
        print(f'Recommendation # {i + 1} : {title} \n')
    
    
    
    
movie_recs(1,df,10)
    

Recommendation # 1 : Usual Suspects, The (1995) 

Recommendation # 2 : Star Wars: Episode IV - A New Hope (1977) 

Recommendation # 3 : Reservoir Dogs (1992) 

Recommendation # 4 : Goodfellas (1990) 

Recommendation # 5 : Matrix, The (1999) 

Recommendation # 6 : Fight Club (1999) 

Recommendation # 7 : Shawshank Redemption, The (1994) 

Recommendation # 8 : Rear Window (1954) 

Recommendation # 9 : Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 

Recommendation # 10 : Godfather, The (1972) 



# Hybrid Filtering